In [1]:
import torch
import torch.nn as nn
from torch.nn import Embedding

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import dataset
import test_lstm

2023-03-10 18:03:24.599457: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
lie_trial_path = './processed_lie/' #60 entries
truth_trial_path = './processed_truth/' #61 entries
MU3D_path = './processed/' # 300 entries

In [3]:
# no split by person
numOfFrames = 10
X, Y = dataset.preprocessing(truth_trial_path, lie_trial_path, numOfFrames=numOfFrames)

TEST_RATIO = 0.2

xTrain, xTest = train_test_split(X, test_size=TEST_RATIO, shuffle=False)
yTrain, yTest = train_test_split(Y, test_size=TEST_RATIO, shuffle=False)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
embedding_layer = Embedding(num_embeddings=numOfFrames, embedding_dim=4)

positions = torch.tensor([0,1,2,3,4,5,6,7,8,9])
embedded_positions = embedding_layer(positions)
embedded_positions

tensor([[ 0.5563, -0.7963, -1.0364, -0.6048],
        [-1.8724, -0.7711,  0.3486,  1.2059],
        [-0.5719, -0.0841, -0.7283,  0.1213],
        [-0.1918, -0.3327,  2.5167,  0.7431],
        [-1.0752, -0.6436, -0.8822, -2.4208],
        [ 0.4202, -1.5169,  0.0819,  0.5593],
        [ 2.0171, -0.4613,  1.5914,  0.2973],
        [ 0.5733, -0.1668,  0.6712, -0.5410],
        [-2.2083, -0.2195,  0.7277, -0.7213],
        [-1.9518, -0.5718,  0.5207,  1.5157]], grad_fn=<EmbeddingBackward0>)

In [6]:
#embedding
def gen_loc_list(max_frames_count, pos_encoder_size, frames_count = numOfFrames):
  embedding = nn.Embedding(max_frames_count, pos_encoder_size)
  input = torch.tensor([i for i in range(frames_count)]).clone().detach()
  loc_list = embedding(input)

  return loc_list

def gen_data(data_arr, embedding_arr):

  data_arr = torch.tensor(data_arr).clone().detach()
  list = torch.cat((data_arr, embedding_arr), 1)

  return list

In [7]:
#model

from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder

class TransformerModel(nn.Module):

  def __init__(self, inFeatCount, num_encoder_layers, n_heads = 5, n_hidden = 500, dropout = 0.3, outFeatCount = 2): #Out = [prob for 0, prob for 1]
    super(TransformerModel, self).__init__()

    encoder_layer = TransformerEncoderLayer(inFeatCount, n_heads, n_hidden, dropout)

    self.encoder = TransformerEncoder(encoder_layer, num_encoder_layers)

    self.decoder = nn.Linear(inFeatCount, outFeatCount)

  def init_weights(self):
      initrange = 0.1
      self.encoder.weight.data.uniform_(-initrange, initrange)
      self.decoder.bias.data.zero_()
      self.decoder.weight.data.uniform_(-initrange, initrange)

  def forward(self, data):
      encoded = self.encoder(data)
      output = self.decoder(encoded)
      return torch.nn.functional.softmax(output)
  
def predict(model, inputArr):
    inputArr = inputArr.to(device).float()
    
    output = model(inputArr)
    
    return output


In [8]:
pos_encoder_size = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
#generate embedding
embedding_arr = gen_loc_list(numOfFrames, pos_encoder_size)

#generate embedded data
x_Train = []
x_Test = []

y_Train = []
y_Test = []

for i in range(xTrain.shape[0]):
    x_Train.append(gen_data(xTrain[i], embedding_arr).detach().numpy())

for i in range(xTest.shape[0]):
    x_Test.append(gen_data(xTest[i], embedding_arr).detach().numpy())

for i in range(yTrain.shape[0]):
    if yTrain[i] == 0:
        y_Train.append([[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0]])
    else:
        y_Train.append([[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]])

for i in range(yTest.shape[0]):
    if yTest[i] == 0:
        y_Test.append([[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0]])
    else:
        y_Test.append([[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]])

#generate model
model = TransformerModel(15, 5).to(device)


In [15]:
# training
import torch.optim as optim


def train(model, xTrain, yTrain, xTest, yTest, epochs = 100, lr = 0.001, batch_size = 10):
    """ Train a model on a dataset """

    loss_items = []
    accuracy_items = []
    test_accuracy_items = []
    
    # create a data loader to handle batching
    xTrain_loader =  torch.utils.data.DataLoader(xTrain, batch_size=batch_size, shuffle=False)
    xTest_loader = torch.utils.data.DataLoader(yTest, batch_size=batch_size, shuffle=False)

    # create a loss function and optimizer
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # train the model
    for epoch in range(epochs):

        # train

        idx = 0
        model.train()
        for batch in xTrain_loader:

            # get data
            x_train = batch.to(device).float()
            y_train = torch.tensor(yTrain[idx]).to(device)

            # forward pass
            y_pred = model(x_train)

            # compute loss
            loss = loss_fn(y_pred, y_train)

            # backward pass
            loss.backward()

            # update weights
            optimizer.step()
            optimizer.zero_grad()

        # evaluate
        model.eval()

        test_acc = 0
        acc = 0
        with torch.no_grad():
            
            
            for batch in xTest_loader:
                y_pred = model(batch.to(device).float())
                y_test = torch.tensor(yTest[idx]).to(device)

                # compute test accuracy
                test_acc += (y_pred.argmax(1) == y_test).type(torch.float).mean().item()
                

            test_acc / len(xTest_loader)  
            test_accuracy_items.append(test_acc)

            for batch in xTrain_loader:
                y_pred = model(batch.to(device).float())
                y_train = torch.tensor(yTrain[idx]).to(device)

                # compute accuracy
                acc += (y_pred.argmax(1) == y_train).type(torch.float).mean().item()

            acc / len(xTrain_loader)
            accuracy_items.append(acc)

        # store loss and accuracy
        loss_items.append(loss.item())

        # print progress
        if (epoch+1) % 1 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}, Accuracy: {acc:.4f}, Test Accuracy: {test_acc:.4f}')

        idx += 1
            

train(model, x_Train, y_Train, x_Test, y_Test)

/var/folders/66/pn8653k51_75x5ksbpsy8hm80000gn/T/ipykernel_22055/476567611.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return torch.nn.functional.softmax(output)


AttributeError: 'list' object has no attribute 'dim'